In [3]:
sc.stop()
import pyspark
from pyspark import SparkContext, SparkConf
sc =SparkContext()
from functools import reduce
import operator as op
import numpy as np

csv_data=sc.textFile("/Users/manasakumar/Desktop/like_proc_train.csv").map(lambda line: line.split(","))
header =csv_data.first() #extract header
csv_data = csv_data.filter(lambda row:row != header)

user_author=csv_data.map(lambda x:(x[0],x[2]))
article_author=csv_data.map(lambda x:(x[1],x[2]))
user_article_rdd=csv_data.map(lambda x:(x[0],x[1])).collect()

user_author_rdd=user_author.mapValues(lambda x:x.split(" "))
article_author_rdd=article_author.mapValues(lambda x:x.split(" ")).collect()

article_count=article_author.map(lambda x: x[0]).distinct().collect()

data1 =user_author_rdd.groupByKey()
user_author_list=data1.map(lambda x: (x[0], list(x[1]))).collect()
user_count=len(user_author_list)

data2=user_author_rdd.combineByKey(lambda value:(value,1),lambda x,value:(x[0]+value,x[1]+1),lambda x,y:(x[0]+y[0],x[1]+y[1]))
user_author_commonlist=data2.collect()

num=[];
lengths=[];
for i in range(0,user_count):
    length=user_author_commonlist[i][1][1]
    lengths.append(length)
    num1=0;
    for j in range(0,length):
        for k in range(j+1,length):
            output=list(set(user_author_list[i][1][j]).intersection(user_author_list[i][1][k]))
            if len(output)>0:
                num1=num1+1
            
    num.append(num1)

def ncr(n, r):
    r = min(r, n-r)
    if r == 0: return 1
    numer = reduce(op.mul, range(n, n-r, -1))
    denom = reduce(op.mul, range(1, r+1))
    return numer/denom

fe1s=[];
for i in range (0,len(lengths)):
    if(num[i]>0):
        den1=ncr(lengths[i],2)
        fe1=num[i]/float(den1)
        
    else:
        fe1=0
    fe1s.append(fe1)
    #print (fe1s[i])


from collections import Counter
fe2s=[]
num_common=[]
for i in range (0,len(lengths)):
    most_common,num_most_common = Counter(user_author_commonlist[i][1][0]).most_common(1)[0]
  
    if(num_most_common>1):
        num_common.append(num_most_common)
    else:
        num_common.append(1)
    
for i in range (0,len(lengths)):  
    fe2=float(num_common[i])/float(lengths[i])
    fe2s.append(fe2)
    #print (fe2s[i])

skim1=[]
count=0;
print max(fe1s)
print max(fe2s)
for i in fe1s:
	count=count+1
	if (i>=1):
		skim1.append(count)
skim2=[]
count=0;
for i in fe2s:
	count=count+1
	if (i>=1):
		skim2.append(count)
        
def intersection(lst1, lst2):
	return list(set(lst1) & set(lst2))

skim_list= intersection(skim1,skim2)
skim = list(map(str,skim_list))
skimmed=csv_data.filter(lambda x: x[0] in (skim))

user_author_skimmed=skimmed.map(lambda x:(x[0],x[2]))
article_author_skimmed=skimmed.map(lambda x:(x[1],x[2]))
user_article_skimmed_rdd=skimmed.map(lambda x:(x[0],x[1])).collect()

user_author_skimmed_rdd=user_author_skimmed.mapValues(lambda x:x.split(" "))
article_author_skimmed_rdd=article_author_skimmed.mapValues(lambda x:x.split(" ")).collect()

article_distinct_skimmed=article_author_skimmed.map(lambda x: int(x[0]) ).distinct().collect()
article_count_skimmed=len(article_distinct_skimmed)
data3 =user_author_skimmed_rdd.groupByKey()
user_author_skimmed_list=data3.map(lambda x: (x[0], list(x[1]))).collect()
user_count_skimmed=len(user_author_skimmed_list)

data4=user_author_skimmed_rdd.combineByKey(lambda value:(value,1),lambda x,value:(x[0]+value,x[1]+1),lambda x,y:(x[0]+y[0],x[1]+y[1]))
user_author_skimmed_commonlist=data4.collect()

import numpy
waa=numpy.zeros((article_count_skimmed,article_count_skimmed),dtype=int)

# finding matrix WAA where WAA(i,j)=WAA(j,i) symmetric matrix Article-Article

for i in range(0,article_count_skimmed):
    for j in range(i,article_count_skimmed):
        output=list(set(article_author_skimmed_rdd[i][1]).intersection(article_author_skimmed_rdd[j][1]))   
        if len(output)>=1:
            waa[j][i]=1
            
#print (waa)


#count=-1
wra=numpy.zeros((user_count_skimmed,article_count_skimmed),dtype=int)  # finding matrix WRA Research-Article
for i in range(0,user_count_skimmed):
    for j in range(0,article_count_skimmed):
        if skim_list[i] == int(user_article_skimmed_rdd[j][0]):
		wra[i][j]=1
#print(wra)
war=wra.transpose()
#print(war)


wrr=numpy.zeros((user_count_skimmed,user_count_skimmed),dtype=int)            #Given in paper Researcher-Researcher is null

sum_ra1=[]   
    
sum_ra=0
for i in range(0,user_count_skimmed):
    sum_ra=0
    for j in range(0,article_count_skimmed):
        sum_ra=sum_ra+wra[i][j]
    sum_ra1.append(sum_ra)
#print(sum_ra1)

sum_ar1=[]   
    
sum_ar=0
for i in range(0,article_count_skimmed):
    sum_ar=0
    for j in range(0,user_count_skimmed):
        sum_ar=sum_ar+war[i][j]
    sum_ar1.append(sum_ar)
#print(sum_ar1)

sum_aa1=[]

sum_aa=0
for i in range(0,article_count_skimmed):
    sum_aa=0  
    for j in range(0,article_count_skimmed):
        sum_aa=sum_aa+waa[i][j]
    sum_aa1.append(sum_aa)
#print(sum_aa1)



tra=numpy.zeros((user_count_skimmed,article_count_skimmed),dtype=float)  # finding matrix WRA Research-Article
numpy.set_printoptions(precision=4)

 #Transition probability moving to an article vertex from researcher
for i in range(0,user_count_skimmed):
    for j in range(0,article_count_skimmed):
        tra[i][j]=wra[i][j]/sum_ra1[i]
#print(tra)



trr=numpy.zeros((user_count_skimmed,user_count_skimmed),dtype=float) 
#print (trr)



tar=numpy.zeros((article_count_skimmed,user_count_skimmed),dtype=float)  
numpy.set_printoptions(precision=4)

for i in range(0,article_count_skimmed):
    for j in range(0,user_count_skimmed):
        tar[i][j]=float(war[i][j])/(float(sum_ar1[i])+float(sum_aa1[i]))
#print(tar)


#Transition probability moving to an article vertex from article vertex

taa=numpy.zeros((article_count_skimmed,article_count_skimmed),dtype=float)  
numpy.set_printoptions(precision=4)

  #Transition probability moving to an article vertex from researcher

for i in range(0,article_count_skimmed):
    for j in range(0,article_count_skimmed):
        taa[i][j]=float(waa[i][j])/(float(sum_ar1[i])+float(sum_aa1[i]))
#print(taa)

t=np.asarray(np.bmat([[trr,tra],[tar,taa]]))
print (t)


g=np.asarray(np.bmat([[wrr,wra],[war,waa]]))
print (g)


#Random walk
#for target researcher vertex- v0


num_vertices=user_count_skimmed+article_count_skimmed

#print (num_vertices)
ScoreArticle=numpy.zeros((article_count_skimmed),dtype=float)
ScoreAll=numpy.zeros((num_vertices),dtype=float)
numpy.set_printoptions(precision=8)
#print(ScoreAll)


print len(skim_list)
print skim_list
all_users_top_20_articles=[]
for index in xrange(len(skim_list)):
    v0=index #0,1,2
    ScoreAll[v0]=1
    #print(ScoreAll)
    maxStep=6
    step=0
    alpha=0.2
    for step in range(maxStep):
        for i in range(num_vertices):
            tmpScore=numpy.zeros((num_vertices),dtype=float)
        for v1 in range(num_vertices):
            for v2 in range(num_vertices):
                tmpScore[v2]=(alpha*ScoreAll[v1]*t[v1][v2])+(tmpScore[v2])
            if (v1==v0):
                tmpScore[v1]=tmpScore[v1]+(1-alpha)
            #print (tmpScore)
        ScoreAll=tmpScore
    #print(ScoreAll)
    ScoreArticle=ScoreAll[user_count_skimmed:user_count_skimmed+article_count_skimmed]


    dic ={k:v for k,v in zip(article_distinct_skimmed,ScoreArticle)}  
    from collections import OrderedDict
    final_list = OrderedDict(sorted(dic.items(),key=lambda kv: kv[1],reverse=True))
  

    counter=0
    user_top_20_articles=[]
    user_top_20_articles.append(skim_list[index])
    user_data=[]
    for i in final_list:
        #print(i," ",dic[i])
        if dic[i]>0.7:
            user_top_20_articles.append(i)
        #counter=counter+1
      
            
    all_users_top_20_articles.append(user_top_20_articles)
        
    #print skim_list[index]
    #print user_top_20_articles
    
#print all_users_top_20_articles
   
import pandas as pd
df_final=pd.DataFrame(all_users_top_20_articles)
#print df_final #first column will be user_id
            
df_final.to_csv('/Users/manasakumar/Desktop/results.csv')





1.0
1.0


/anaconda2/lib/python2.7/site-packages/ipykernel_launcher.py:181: RuntimeWarning: divide by zero encountered in long_scalars


[[0.     0.     0.     ... 0.     0.     0.    ]
 [0.     0.     0.     ... 0.     0.     0.    ]
 [0.     0.     0.     ... 0.     0.     0.    ]
 ...
 [0.     0.     0.     ... 0.0008 0.     0.    ]
 [0.     0.     0.     ... 0.0009 0.0009 0.    ]
 [0.     0.     0.     ... 0.0009 0.0009 0.0009]]
[[0 0 0 ... 0 0 0]
 [0 0 0 ... 0 0 0]
 [0 0 0 ... 0 0 0]
 ...
 [0 0 0 ... 1 0 0]
 [0 0 0 ... 1 1 0]
 [0 0 0 ... 1 1 1]]
99
[1539, 5126, 5128, 3596, 5135, 3089, 3093, 2588, 2078, 4642, 1064, 1577, 1580, 4142, 2096, 51, 2615, 2106, 59, 4156, 2628, 3145, 76, 1114, 5215, 4211, 118, 127, 3716, 2191, 5782, 4253, 2724, 3364, 2221, 4282, 2237, 2750, 4813, 1743, 2259, 2265, 4319, 3808, 3298, 235, 5365, 251, 2812, 2815, 5891, 2321, 1298, 5398, 4897, 804, 3880, 813, 814, 4401, 1347, 2376, 3914, 3917, 2382, 910, 2391, 2393, 1379, 5480, 3470, 6003, 3445, 888, 4987, 1411, 4999, 3466, 909, 5006, 413, 4000, 2981, 424, 3512, 4084, 1978, 2499, 3524, 1485, 3033, 474, 4572, 477, 5088, 1011, 500, 3068, 5973]
